In [229]:
text_list = ['อาหารอร่อยมากเลยครับ','ชอบรายการอาหารที่หลากหลาย','มีกลิ่นติดตัว', 'รายการอาหารไม่มากพอ' , 'บริการดีมาก', 'พนักงานไม่เอาใจใส่']

In [230]:
import re
import string

def clean_msg(msg):
    
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ …
    msg = re.sub(r'…','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [231]:
print('original text:\n',text_list[0])
print('clean text:\n',clean_msg(text_list[0]))

original text:
 อาหารอร่อยมากเลยครับ
clean text:
 อาหารอร่อยมากเลยครับ


In [232]:
print('original text:\n',text_list[1])
print('clean text:\n',clean_msg(text_list[1]))

original text:
 ชอบรายการอาหารที่หลากหลาย
clean text:
 ชอบรายการอาหารที่หลากหลาย


In [233]:
clean_text = [clean_msg(txt) for txt in text_list]

In [234]:
#!pip install pythainlp
#!pip install stop_words
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words

In [235]:
import nltk
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

#print(th_stop)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\George\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [236]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    #tokens = [i for i in tokens ]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [237]:
print('tokenized text:\n',split_word(clean_msg(text_list[0])))
print('tokenized text:\n',split_word(clean_msg(text_list[1])))
print('tokenized text:\n',split_word(clean_msg(text_list[2])))
print('tokenized text:\n',split_word(clean_msg(text_list[3])))
print('tokenized text:\n',split_word(clean_msg(text_list[4])))

tokenized text:
 ['อาหาร', 'อร่อย']
tokenized text:
 ['ชอบ', 'เมนู', 'หลากหลาย']
tokenized text:
 ['กลิ่น', 'ติดตัว']
tokenized text:
 ['เมนู']
tokenized text:
 ['บริการ', 'ดีมาก']


In [238]:
tokens_list = [split_word(txt) for txt in clean_text]

In [239]:
tokens_list

[['อาหาร', 'อร่อย'],
 ['ชอบ', 'เมนู', 'หลากหลาย'],
 ['กลิ่น', 'ติดตัว'],
 ['เมนู'],
 ['บริการ', 'ดีมาก'],
 ['พนักงาน', 'เอาใจใส่']]

In [240]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [241]:
vocab = cvec.vocabulary_
print(vocab)

{'อาหาร': 8, 'อร่อย': 7, 'ชอบ': 1, 'เมนู': 9, 'หลากหลาย': 6, 'กลิ่น': 0, 'ติดตัว': 3, 'บริการ': 4, 'ดีมาก': 2, 'พนักงาน': 5, 'เอาใจใส่': 10}


In [242]:
c_feat[:,:20].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]], dtype=int64)

In [243]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
t_feat = tvec.fit_transform(tokens_list_j)

In [244]:
t_feat[:,:5].todense()

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.61171251, 0.        , 0.        , 0.        ],
        [0.70710678, 0.        , 0.        , 0.70710678, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.70710678, 0.        , 0.70710678],
        [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [245]:
print(len(tvec.idf_),len(tvec.vocabulary_))

11 11


In [246]:
c_feat[:,:5].todense()

matrix([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0]], dtype=int64)

In [247]:
print(c_feat.todense())

[[0 0 0 0 0 0 0 1 1 0 0]
 [0 1 0 0 0 0 1 0 0 1 0]
 [1 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1]]


In [248]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

In [249]:
X = np.array(c_feat.todense())

In [250]:
X

array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]], dtype=int64)

In [251]:
 y = np.array(['+', '+', '-', '-','+','-'])

In [252]:
clf = MultinomialNB(class_prior=[0.25, 0.5])

# Train model
model = clf.fit(X, y)

In [253]:
# Create new observation
new_observation = [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [254]:
# Predict new observation's class
y_predicted = model.predict(X)

In [255]:
print(y_predicted)

['+' '+' '-' '-' '+' '-']
